# Salt Identification Challenge
## Capstone Project
Shane Moloney  
Udacity Machine Learning Nanodegree

## Set up

Imports for all the project dependencies.
I may split this so that the dependencies are in the relevant code cells to avoid needing to run this every time.

In [ ]:
import numpy as np
import pandas as pd
import math

from random import randint

import cv2
from pprint import pprint

import matplotlib.pyplot as plt
plt.style.use('seaborn-white')
import seaborn as sns
sns.set_style("white")

from sklearn.model_selection import train_test_split

from skimage.transform import resize

from keras.preprocessing.image import load_img
from keras import Model
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.models import load_model
from keras.optimizers import Adam
from keras.utils.vis_utils import plot_model
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Input, Conv2D, Conv2DTranspose, MaxPooling2D, concatenate, Dropout

from tqdm import tqdm_notebook

### Load in a Sample of the data

Let's have a look at the data to see what will need to be done to prepare it for learning.

In [ ]:
train_samp = pd.read_csv("../input/train.csv", index_col="id", nrows=5)
depths_samp = pd.read_csv("../input/depths.csv", index_col="id", nrows=5)
print(train_samp)
print(depths_samp)

Above we can see the run-length encoded masks and the depths of the first five images.  

As we can see, images with no salt deposits have a null value and images with deposits have the pixels listed as ranges that cantain salt.  

Now we will look at the images themselves along with their provided masks as these are kindly provided to us in png form although they could also be extracted from the train.csv file by decoding the entries in the file.

In [ ]:
train_inds = train_samp.index

image_samp = [np.array(load_img("../input/train/images/{}.png".format(idx), grayscale=True)) / 255 for idx in train_inds]
mask_samp = [np.array(load_img("../input/train/masks/{}.png".format(idx), grayscale=True)) / 255 for idx in train_inds]
image_size = image_samp[0].shape
print("Image Dimensions: {}".format(image_size))

fig, axs = plt.subplots(5, 2, figsize=(10, 25))
for i in range(0, 5):
    axs[int(i)][0].imshow(image_samp[i-1], cmap='gray')
    axs[int(i)][1].imshow(mask_samp[i-1], cmap='gray')
plt.show()


We can see that some masks may not be accurate as in the figure above the mask seems to claim that the salt deposit takes up exactly half the image with a perfectly straight border, which we can clearly see is incorrect.  

I may decide to remove any data whose mask matches this pattern.  

For now lets load in all the data and start looking at it as a whole.

## Loading in the Data

First we'll import the CSV files and add the depths as well and store a list of the indeces and a list of the test set depths, ie the depths whose ID's do not appear in the test set.

In [ ]:
train_full = pd.read_csv("../input/train.csv", index_col="id", usecols=[0])
depths = pd.read_csv("../input/depths.csv", index_col="id")

train_full = train_full.join(depths)
train_idx = train_full.index

test_depths = depths[~depths.index.isin(train_full.index)]



Now we import the image and mask files, 4000 in total.

In [ ]:
train_full["images"] = [np.array(load_img("../input/train/images/{}.png".format(idx), grayscale=True)) / 255 for idx in tqdm_notebook(train_idx)]

In [ ]:
train_full["masks"] = [np.array(load_img("../input/train/masks/{}.png".format(idx), grayscale=True)) / 255 for idx in tqdm_notebook(train_idx)]

Create a column in the array for the coverage of the salt in each image. This means the number of pixels xcontaining salt over the area of the image. As this can be any decimal between 0 and 1 we discretise these values with the cov_to_class function which gives it an integer class from 1 to 10 which represent no salt up to only salt respectively.

In [ ]:
train_full["coverage"] = train_full.masks.map(np.sum) / pow(image_size[0], 2)

In [ ]:
def cov_to_class(val):    
    for i in range(0, 11):
        if val * 10 <= i :
            return i
        
train_full["coverage_class"] = train_full.coverage.map(cov_to_class)

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(15,5))
sns.distplot(train_full.coverage, kde=False, ax=axs[0])
sns.distplot(train_full.coverage_class, bins=10, kde=False, ax=axs[1])
plt.suptitle("Salt coverage")
axs[0].set_xlabel("Coverage")
axs[1].set_xlabel("Coverage class")

We can see that a large amount of the images have no or very little salt, half the data set resides in classes 1 and 2. This is good as the model needs to know that there are images without salt as well.

## Depths
Now let's have a look at the depths and how good their random sampling was.

In [ ]:
sns.distplot(train_full.z, label="Train")
sns.distplot(test_depths.z, label="Test")
plt.legend()
plt.title("Depth distribution")

A nice gaussian distribution was produced on both the train and testing sets which means we'll get a good range of different sediment layers.

## Preprocessing

First I'm, going to set the masks to all black for any that have perfectly straight borders as these are definitely not accurate and rather than remove them I will simply set them to a no salt state.

In [ ]:
# I use opencv to edge detect the masks
# As the images are segmented by row I rotate the edge image
# I check if the row is entirely filled with an edge meaning a perfectly straight edge
def has_straight_border(mask_id, shape, edge_array):
    mask = cv2.imread('../input/train/masks/{}.png'.format(mask_id))
    edges = cv2.Canny(mask, 100, 200)
    mat = cv2.getRotationMatrix2D((shape[0]/2,shape[1]/2),90,1)
    edges_rotated = cv2.warpAffine(edges, mat, shape)
    edge_array.append(edges_rotated)
    for row in edges_rotated:
        if all(pix == 255 for pix in row):
            return True
    return False

# Now find the list of straight edged masks
straight_borders = []
edges = []
for idx in train_idx:
    if has_straight_border(idx, image_size, edges):
        straight_borders.append(idx)
print("{} masks with straight borders.".format(len(straight_borders)))

Now I'll have a look at the resulting masks and see if they make sense to remove.
I'll do this by overlaying the masks onto the images, where the salt is in green and the other sediment is in grey.

In [ ]:
max_images = 117
grid_width = 9
grid_height = int(max_images / grid_width)
fig, axs = plt.subplots(grid_height, grid_width, figsize=(grid_width*2, grid_height*2))
for i, idx in enumerate(straight_borders[:max_images]):
    mask = train_full.loc[idx].masks
    ax = axs[int(i / grid_width), i % grid_width]
    ax.imshow(train_full.images[np.where(train_idx == idx)[0][0]], cmap="Greys")
    ax.imshow(mask, alpha=0.3, cmap="Greens")
    ax.set_yticklabels([])
    ax.set_xticklabels([])

There are 90 masks with perfectly straight borders, some of which look to be just inaccurate labelling, others seem to be mostly salt with colums of sediment. I don't want to lose the latter information as we saw in the coverage graph many of the images are quite low salt coverage. To remedy this I am going to keep any data points with a coverage class of 8 and above and drop the rest so as to avoid innaccurate border definition in the model.

In [ ]:
full_len = len(straight_borders)
to_be_removed = []
for idx in straight_borders:
    if train_full.coverage_class[idx] < 8:
        to_be_removed.append(idx)
        
print("{}/{} stright bordered data points to be removed:".format(len(to_be_removed), full_len))
pprint(to_be_removed)

In [ ]:
train_full = train_full.drop(to_be_removed)
train_idx = train_full.index
print("New Data Shape: {}".format(train_full.shape))

## Creating the Training and Validation Sets
I'll use the coverage class attribute as a stratification criterion to allow for even distribution in both sets.  

First we'll need a function to resize images to be an exponentially divisible number for the Unet model.
As well as a function for returning the images back to their original size.

In [ ]:
new_size = 128

# Resizes the image to a square with sides the equal to the size provided
def upsample(img):
    if img.shape[0] == new_size:
        return img
    return resize(img, (new_size, new_size), mode='constant', preserve_range=True)

# Will need to return images to their original size for post processing
def downsample(img):
    if img.shape[0] == image_size[0]:
        return img
    return resize(img, image_size, mode='constant', preserve_range=True)

In [ ]:
train_id, val_id, train_x, val_x, train_y, val_y, train_cov, test_cov, train_depth, test_depth = train_test_split(
    train_idx,
    np.array(train_full.images.map(upsample).tolist()).reshape(-1, new_size, new_size, 1), 
    np.array(train_full.masks.map(upsample).tolist()).reshape(-1, new_size, new_size, 1), 
    train_full.coverage.values,
    train_full.z.values,
    test_size=0.2, stratify=train_full.coverage_class, random_state=117)

In [ ]:
tmp_img = np.zeros((new_size, new_size), dtype=train_full.images[train_id[25]].dtype)
tmp_img[:image_size[0], :image_size[1]] = train_full.images[train_id[25]]
fix, axs = plt.subplots(1, 2, figsize=(15,5))
axs[0].imshow(tmp_img, cmap="Greys")
axs[0].set_title("Original image")
axs[1].imshow(train_x[25].squeeze(), cmap="Greys")
axs[1].set_title("Scaled image")

Looks like the splitting and scaling has worked perfectly.

## Augmenting the Data

I will also augment the data to horizontally flip the images and add these to the current images doubling our training data set as the nature of the images and their non-uniformity allows us to do this without risking overfitting.

In [ ]:
train_x = np.append(train_x, [np.fliplr(img) for img in train_x], axis=0)
train_y = np.append(train_y, [np.fliplr(img) for img in train_y], axis=0)

In [ ]:
fig, axs = plt.subplots(2, 10, figsize=(15, 3))
for i in range(10):
    axs[0][i].imshow(train_x[i].squeeze(), cmap="Greys")
    axs[0][i].imshow(train_y[i].squeeze(), cmap="Greens", alpha=0.3)
    axs[1][i].imshow(train_x[int(len(train_x)/2 + i)].squeeze(), cmap="Greys")
    axs[1][i].imshow(train_y[int(len(train_x)/2 + i)].squeeze(), cmap="Greens", alpha=0.3)

## Building the Model

Now I need to build the Unet model, I'm going to make a function for this so that I can experiment with different amounts of channels. Because of how Unets work I am limited in the number of layers I can create based on the image size.

In [ ]:
def build_model(input_layer, base_channels):
    dropout = 0.25
    kernel = (3, 3)
    stride = (2, 2)
    # Image size: 128 to 64
    conv1 = Conv2D(base_channels, kernel, activation="relu", padding="same")(input_layer)
    conv1 = Conv2D(base_channels, kernel, activation="relu", padding="same")(conv1)
    pool1 = MaxPooling2D((2, 2))(conv1)
    pool1 = Dropout(dropout)(pool1)
    
    # Image size: 64 to 32
    conv2 = Conv2D(base_channels * 2, kernel, activation="relu", padding="same")(pool1)
    conv2 = Conv2D(base_channels * 2, kernel, activation="relu", padding="same")(conv2)
    pool2 = MaxPooling2D((2, 2))(conv2)
    pool2 = Dropout(dropout*2)(pool2)
    
    # Image size: 32 to 16
    conv3 = Conv2D(base_channels * 4, kernel, activation="relu", padding="same")(pool2)
    conv3 = Conv2D(base_channels * 4, kernel, activation="relu", padding="same")(conv3)
    pool3 = MaxPooling2D((2, 2))(conv3)
    pool3 = Dropout(dropout*2)(pool3)
    
    # Image size: 16 to 8
    conv4 = Conv2D(base_channels * 8, kernel, activation="relu", padding="same")(pool3)
    conv4 = Conv2D(base_channels * 8, kernel, activation="relu", padding="same")(conv4)
    pool4 = MaxPooling2D((2, 2))(conv4)
    pool4 = Dropout(dropout*2)(pool4)
    
    # Midway point
    conv_mid = Conv2D(base_channels * 16, kernel, activation="relu", padding="same")(pool4)
    conv_mid = Conv2D(base_channels * 16, kernel, activation="relu", padding="same")(conv_mid)
    
    # Now we mirror the above layers and begin increasing the image size using Conv2DTranspose layers
    # Image size: 8 to 16
    trans_conv4 = Conv2DTranspose(base_channels * 8, kernel, strides=stride, padding="same")(conv_mid)
    uconv4 = concatenate([trans_conv4, conv4])
    uconv4 = Dropout(dropout * 2)(uconv4)
    uconv4 = Conv2D(base_channels * 8, kernel, activation="relu", padding="same")(uconv4)
    uconv4 = Conv2D(base_channels * 8, kernel, activation="relu", padding="same")(uconv4)
    
    # Image size: 16 to 32
    trans_conv3 = Conv2DTranspose(base_channels * 4, kernel, strides=stride, padding="same")(uconv4)
    uconv3 = concatenate([trans_conv3, conv3])
    uconv3 = Dropout(dropout * 2)(uconv3)
    uconv3 = Conv2D(base_channels * 4, kernel, activation="relu", padding="same")(uconv3)
    uconv3 = Conv2D(base_channels * 4, kernel, activation="relu", padding="same")(uconv3)
    
    # Image size: 32 to 64
    trans_conv2 = Conv2DTranspose(base_channels * 2, kernel, strides=stride, padding="same")(uconv3)
    uconv2 = concatenate([trans_conv2, conv2])
    uconv2 = Dropout(dropout * 2)(uconv2)
    uconv2 = Conv2D(base_channels * 2, kernel, activation="relu", padding="same")(uconv2)
    uconv2 = Conv2D(base_channels * 2, kernel, activation="relu", padding="same")(uconv2)
    
    # Image size: 64 to 128
    trans_conv1 = Conv2DTranspose(base_channels, kernel, strides=stride, padding="same")(uconv2)
    uconv1 = concatenate([trans_conv1, conv1])
    uconv1 = Dropout(dropout)(uconv1)
    uconv1 = Conv2D(base_channels, kernel, activation="relu", padding="same")(uconv1)
    uconv1 = Conv2D(base_channels, kernel, activation="relu", padding="same")(uconv1)
    
    output_layer = Conv2D(1, (1, 1), padding="same", activation="sigmoid")(uconv1)
    
    return output_layer

In [ ]:
input_layer = Input((new_size, new_size, 1))
output_layer = build_model(input_layer, 16)

In [ ]:
model = Model(input_layer, output_layer)

In [ ]:
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
model.summary()

## Training the Model

Now I can begin training the model, I am unsure what parameters I will start out with but I will experiment with each to find what works best.

I know that my setup will be a large number of epochs so that I can use early stopping to find the best possible model. I will store the best model in a seperate file for later use.

In [ ]:
early_stop = EarlyStopping(patience=10, verbose=1)
model_check = ModelCheckpoint("./keras.model", save_best_only=True, verbose=1)
learning_rate_control = ReduceLROnPlateau(factor=0.1, patience=5, min_lr=0.00001, verbose=1)
epochs = 100
batch_size = 32

results = model.fit(train_x, train_y,
                    validation_data=[val_x, val_y],
                    epochs=epochs,
                    batch_size=batch_size,
                    callbacks=[early_stop, model_check, learning_rate_control])

Now let's have a look at how the training loss and the validation loss values compare as well as the training and validation accuracy.  
We should be able to easily see the point at which the two begin to diverge which is where our best model lies.

In [ ]:
fig, (ax_loss, ax_acc) = plt.subplots(1, 2, figsize=(15,5))
ax_loss.plot(results.epoch, results.history["loss"], label="Train loss")
ax_loss.plot(results.epoch, results.history["val_loss"], label="Validation loss")
ax_acc.plot(results.epoch, results.history["acc"], label="Train accuracy")
ax_acc.plot(results.epoch, results.history["val_acc"], label="Validation accuracy")

In [ ]:
model = load_model("./keras.model")

## Sanity Check using Validation Set
I'll have the model predict the validation set and overlay the prediction onto the images and masks to see how they match up.

In [ ]:
val_predictions = model.predict(val_x).reshape(-1, new_size, new_size)
val_predictions = np.array([downsample(x) for x in val_predictions])
val_y_original = np.array([train_full.masks[idx] for idx in val_id])

In [ ]:
max_images = 60
grid_width = 15
grid_height = int(max_images / grid_width)
fig, axs = plt.subplots(grid_height, grid_width, figsize=(grid_width, grid_height))
for i, idx in enumerate(val_id[:max_images]):
    img = train_full.images[idx]
    mask = train_full.masks[idx]
    pred = val_predictions[i]
    ax = axs[int(i / grid_width), i % grid_width]
    ax.imshow(img, cmap="Greys")
    ax.imshow(mask, alpha=0.3, cmap="Greens")
    ax.imshow(pred, alpha=0.3, cmap="OrRd")
    ax.set_yticklabels([])
    ax.set_xticklabels([])
plt.suptitle("Green: salt, Red: prediction.")

The intensity of the red represents the probability that the pixel is salt as predicted by the model.
Some of the images have been predicted erfectly with exact borders between salt and sediment, some images with large sections of salt the model struggles to register all of it. This leads me to believe the model has perhaps trained to recognise the border quite well rather than learning to recognise the salt itself.
The fuzzier edges and sections are where the model predicts a low probablility of salt.  

Now we'll run the IoU metrics with each provided threshold and graph the score received at each threshold.

In [ ]:
# src: https://www.kaggle.com/aglotero/another-iou-metric
def iou_metric(y_true_in, y_pred_in, print_table=False):
    labels = y_true_in
    y_pred = y_pred_in
    
    true_objects = 2
    pred_objects = 2

    intersection = np.histogram2d(labels.flatten(), y_pred.flatten(), bins=(true_objects, pred_objects))[0]

    # Compute areas (needed for finding the union between all objects)
    area_true = np.histogram(labels, bins = true_objects)[0]
    area_pred = np.histogram(y_pred, bins = pred_objects)[0]
    area_true = np.expand_dims(area_true, -1)
    area_pred = np.expand_dims(area_pred, 0)

    # Compute union
    union = area_true + area_pred - intersection

    # Exclude background from the analysis
    intersection = intersection[1:,1:]
    union = union[1:,1:]
    union[union == 0] = 1e-9

    # Compute the intersection over union
    iou = intersection / union

    # Precision helper function
    def precision_at(threshold, iou):
        matches = iou > threshold
        true_positives = np.sum(matches, axis=1) == 1   # Correct objects
        false_positives = np.sum(matches, axis=0) == 0  # Missed objects
        false_negatives = np.sum(matches, axis=1) == 0  # Extra objects
        tp, fp, fn = np.sum(true_positives), np.sum(false_positives), np.sum(false_negatives)
        return tp, fp, fn

    # Loop over IoU thresholds
    prec = []
    if print_table:
        print("Thresh\tTP\tFP\tFN\tPrec.")
    for t in np.arange(0.5, 1.0, 0.05):
        tp, fp, fn = precision_at(t, iou)
        if (tp + fp + fn) > 0:
            p = tp / (tp + fp + fn)
        else:
            p = 0
        if print_table:
            print("{:1.3f}\t{}\t{}\t{}\t{:1.3f}".format(t, tp, fp, fn, p))
        prec.append(p)
    
    if print_table:
        print("AP\t-\t-\t-\t{:1.3f}".format(np.mean(prec)))
    return np.mean(prec)

def iou_metric_batch(y_true_in, y_pred_in):
    batch_size = y_true_in.shape[0]
    metric = []
    for batch in range(batch_size):
        value = iou_metric(y_true_in[batch], y_pred_in[batch])
        metric.append(value)
    return np.mean(metric)

In [ ]:
thresholds = np.linspace(0, 1, 50)
ious = np.array([iou_metric_batch(val_y_original, np.int32(val_predictions > threshold)) for threshold in tqdm_notebook(thresholds)])

In [ ]:
threshold_best_index = np.argmax(ious[9:-10]) + 9
iou_best = ious[threshold_best_index]
threshold_best = thresholds[threshold_best_index]

In [ ]:
plt.plot(thresholds, ious)
plt.plot(threshold_best, iou_best, "xr", label="Best threshold")
plt.xlabel("Threshold")
plt.ylabel("IoU")
plt.title("Threshold vs IoU ({}, {})".format(threshold_best, iou_best))
plt.legend()

The graph shows a wide curve where the low threesholds produces a low scre due to very low probability pixels are counted as salt predictions.  
The high thresholds also score lower as only pixels the model is very sure of are counted and so this leaves very little room for error.
As we can see the model does best when the threshold is set to only count pixels as salt predictions with an IoU of ~0.4 -> 0.7.
The goal is to make this curve as tall and as wide as possible since the end metric is the average of all these scores.

In [ ]:
max_images = 60
grid_width = 15
grid_height = int(max_images / grid_width)
fig, axs = plt.subplots(grid_height, grid_width, figsize=(grid_width, grid_height))
for i, idx in enumerate(val_id[:max_images]):
    img = train_full.images[idx]
    mask = train_full.masks[idx]
    pred = val_predictions[i]
    ax = axs[int(i / grid_width), i % grid_width]
    ax.imshow(img, cmap="Greys")
    ax.imshow(mask, alpha=0.3, cmap="Greens")
    ax.imshow(np.array(np.round(pred > threshold_best), dtype=np.float32), alpha=0.3, cmap="OrRd")
    ax.set_yticklabels([])
    ax.set_xticklabels([])
plt.suptitle("Green: salt, Red: prediction.")

With the best scoring threshold applied we can see that the fuzzy edges are significantly reduced, resulting in sharper, more clear predictions.

## Scoring
Now to create the submission file, ads this is a competition with over a thousand participants there was no need to reinvent the wheel and write my own functions for this, I am using the functions created by user bguberfain, link to the source is in the code.

In [ ]:
# Source https://www.kaggle.com/bguberfain/unet-with-depth
def RLenc(img, order='F', format=True):
    """
    img is binary mask image, shape (r,c)
    order is down-then-right, i.e. Fortran
    format determines if the order needs to be preformatted (according to submission rules) or not

    returns run length as an array or string (if format is True)
    """
    bytes = img.reshape(img.shape[0] * img.shape[1], order=order)
    runs = []  ## list of run lengths
    r = 0  ## the current run length
    pos = 1  ## count starts from 1 per WK
    for c in bytes:
        if (c == 0):
            if r != 0:
                runs.append((pos, r))
                pos += r
                r = 0
            pos += 1
        else:
            r += 1

    # if last run is unsaved (i.e. data ends with 1)
    if r != 0:
        runs.append((pos, r))
        pos += r
        r = 0

    if format:
        z = ''

        for rr in runs:
            z += '{} {} '.format(rr[0], rr[1])
        return z[:-1]
    else:
        return runs

Now to load and predict the test set, and create the submission file.

In [ ]:
x_test = np.array([upsample(np.array(load_img("../input/test/images/{}.png".format(idx), grayscale=True))) / 255 for idx in tqdm_notebook(test_depths.index)]).reshape(-1, new_size, new_size, 1)

In [ ]:
preds_test = model.predict(x_test)

In [ ]:
pred_dict = {idx: RLenc(np.round(downsample(preds_test[i]) > threshold_best)) for i, idx in enumerate(tqdm_notebook(test_depths.index.values))}

In [ ]:
sub = pd.DataFrame.from_dict(pred_dict,orient='index')
sub.index.names = ['id']
sub.columns = ['rle_mask']
sub.to_csv('submission.csv')